Iván Ricardo Paredes Avilez
A01705083

12/11/2023

Nombre del archivo: CleaningRobotModel.ipynb

M1.Actividad: Robot de limpieza

Implementación de Sistemas Multiagentes



In [17]:
# Se importan las clases "Agent" y "Model", requeridas para manejar los agentes.
# Además, cada modelo puede tener múltiples agentes
from mesa import Agent, Model 

# "MultiGrid" proporciona una cuadrícula que puede contener múltiples agentes en una sola celda
from mesa.space import MultiGrid

# Para simular que todos los agentes se activen al mismo tiempo, importamos "RandomActivation"
from mesa.time import RandomActivation

# Con "DataCollector" podemos obtener información por cada paso en la simulación
from mesa.datacollection import DataCollector

# Para obtener un mejor manejo de valores numéricos importamos los siguientes paquetes:
import numpy as np
import pandas as pd

# Paquetes utilizados para medir el tiempo de ejecución:
import time
import datetime

# Funcionalidad para trabajar con números aleatorios
import random

In [18]:
class CleaningRobotAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

    def step(self):
        x, y = self.pos
        
        # Verifica si la celda en la que se encuentra el agente está limpia
        if self.model.is_clean(x, y):
            self.move()
        else:
            # Si la celda está sucia, la limpia y luego se mueve
            self.model.clean(x, y)
            self.move()

    def move(self):
        # Obtiene los movimientos posibles alrededor del agente
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Filtra los movimientos posibles para asegurarse de que las celdas estén vacías
        possible_steps = [i for i in possible_steps if self.model.grid.is_cell_empty(i)]
        
        if possible_steps:
            # Si hay movimientos posibles, elige uno al azar y se mueve a esa posición
            new_position = random.choice(possible_steps)
            self.model.grid.move_agent(self, new_position)

In [19]:
class CleaningRobotModel(Model):
    def __init__(self, width, height, num_agents, dirty_cells):

        # Creación de una cuadrícula 2D
        self.grid = MultiGrid(width, height, torus=False)
        # Programación de la activación aleatoria de agentes
        self.schedule = RandomActivation(self)
        # Matriz para rastrear celdas sucias
        self.dirty = np.zeros((width, height))
        # Contador de celdas sucias inicializadas en 0
        self.dirty_cells = 0
        # Contador de celdas limpias inicializado en 0
        self.clean_cells = 0
        # Contador de pasos de simulación inicializado en 0
        self.steps = 0  

        # Creación de agentes y asignación a la cuadrícula
        id = 0
        for i in range(num_agents):
            agent = CleaningRobotAgent(id, self)
            # Agregar agente a la programación de agentes
            self.schedule.add(agent)
            # Todos los agentes empiezan en la celda [1,1].
            self.grid.place_agent(agent, (1, 1))
            id += 1

        total_cells = width * height

        # Cálculo del número total de celdas sucias
        self.dirty_cells = int(total_cells * dirty_cells)

        # Colocación aleatoria de celdas sucias en la cuadrícula
        for i in range(self.dirty_cells):
            # Obtener una celda vacía aleatoria
            x, y = self.random_empty_cell()
            # Marcar la celda como sucia
            self.dirty[x][y] = 1

    def step(self):
        # Avanzar un paso en la simulación
        self.schedule.step()
        # Aumentar el contador de pasos
        self.steps += 1  

    def clean(self, x, y):
        # Marcar la celda como limpia
        self.dirty[x][y] = 0
        # Aumentar el contador de celdas limpias
        self.clean_cells += 1

    def is_clean(self, x, y):
        # Verificar si la celda está limpia
        return self.dirty[x][y] == 0

    def random_empty_cell(self):
        # Obtener una posición aleatoria
        x, y = self.random_pos()

        # Encontrar una celda vacía y no sucia
        while not self.grid.is_cell_empty((x, y)) or self.dirty[x][y] == 1:
            # Obtener otra posición aleatoria
            x, y = self.random_pos()
        return x, y

    # Obtener una posición aleatoria en la cuadrícula
    def random_pos(self):
        return random.randrange(self.grid.width), random.randrange(self.grid.height)

    # Obtener el número de pasos de simulación
    def get_steps(self):
        return self.steps

    # Obtener el número total de celdas sucias
    def get_dirty_cells(self):
        return self.dirty_cells  

    #Obtener el número total de celdas limpias
    def get_clean_cells(self):
        return self.clean_cells

    # Calcular el porcentaje de celdas sucias en la cuadrícula
    def get_dirty_percentage(self):
        total_cells = self.grid.width * self.grid.height
        dirty_cells_count = np.sum(self.dirty)
        dirty_percentage = (dirty_cells_count / total_cells) * 100
        return round(dirty_percentage, 2)

In [20]:
# Limites de la simulación
ANCHO = 100
ALTO = 100
NUM_AGENTES = 10
PORCENTAJE_SUCIO = 0.9 # Ejemplo: 90% = 0.9
MAX_STEPS = 10000 # (Tiempo máximo de ejecución)

# Registro del tiempo de inicio de la simulación
start_time = time.time()

# Creación del modelo de limpieza con parámetros definidos
model = CleaningRobotModel(ANCHO, ALTO, NUM_AGENTES, PORCENTAJE_SUCIO)

# Mostrar el porcentaje inicial de suciedad
print("Porcentaje inicial de suciedad: ", model.get_dirty_percentage())

# Bucle que simula la limpieza hasta que se limpia toda la cuadrícula o se alcanza el máximo de pasos (Tiempo máximo de ejecución)
# for i in range(MAX_STEPS):
    # Verificar si no hay suciedad en la cuadrícula
    # if model.get_dirty_percentage() == 0:
        # break
    # model.step()

# Bucle que se ejecuta hasta que no haya suciedad
while model.get_dirty_percentage() > 0:
    model.step()

# Calcular el tiempo total de simulación
end_time = time.time() - start_time

# Mostrar el porcentaje final de suciedad, el tiempo tomado y el número de pasos realizados en la simulación
print("Porcentaje final de suciedad: ", model.get_dirty_percentage())
print("Tiempo tomado: ", round(end_time, 3), "segundos")
print("Número de pasos: ", model.get_steps())

Porcentaje inicial de suciedad:  90.0
Porcentaje final de suciedad:  0.0
Tiempo tomado:  5.109 segundos
Número de pasos:  27892
